# Iteration 4: Propulsion Weight

In [1]:
from unyt import km, m, mm, inch, g, kg, hr, minute, s, degree, radian, volt
import numpy as np
from matplotlib import pyplot as plt
from math import pi as π
%matplotlib inline

In [2]:
rpm = minute**-1

In [3]:
ρ = 1.225*kg/m**3 # air density
μinf = 1.81e-5*kg/(m*s) # viscosity of air

Goals:
- Use more accurate weight estimate with new power system components

## Initial Design

In [4]:
b = 3050*mm # wingspan
c = 230*mm # chord

In [5]:
S = b*c # wing area
AR = b**2/S

In [6]:
AR

unyt_quantity(13.26086957, '(dimensionless)')

## Weight Estimate

Weight estimate increased from 2340 to 3047 (increased weight of battery, ESC, motor, and future expansion + added prop weight estimate).

In [7]:
W = 3047*g # weight

![iteration_3-weight-tabulation](iteration_3-weight-tabulation.png)

## Airfoil Selection

Cruise speed lowered from 45 to 30.

In [8]:
V = 30*km/hr # cruise speed

In [9]:
CL = (2*W)/(ρ*V**2*S)

In [10]:
Re = (ρ*V*c)/μinf
round(Re.to_value(), -3)

130000.0

![clarky-cl-v-alpha](clarky-cl-v-alpha.png)

From bottom left up: Blue = 50,000  Orange = 100,000  Green = 200,000

In [11]:
α0 = -3.75*degree
Clα = 1.05
e = 0.8

In [12]:
CLα = Clα/(1+(Clα/(π*e*AR)))

In [13]:
CL_unitless = CL.to_value('s**2/m')

In [14]:
α = α0+((CL_unitless/CLα)*radian)
α

unyt_quantity(1.99785477, 'degree')

![clarky-cl-cd-v-alpha](clarky-cl-cd-v-alpha.png)

By increasing the weight our cruise angle of attack increased to 2° which is a lot closer to the optimal. That's a nice consequence.

In [15]:
CLmax = 1.4*s**2/m
Vstall = np.sqrt((2*W)/(ρ*S*CLmax))
print(f"Cruise speed: {V.to('m/s'):.2f}, Stall speed: {Vstall:.2f}")

Cruise speed: 8.33 m/s, Stall speed: 2.25 m/s


Worse than the 1.97 m/s of iteration 3 but still OK.

## Wing Loading and Thrust to Weight Ratio

In [16]:
WCL = W/(S**(3/2))
print(f"Wing loading: {WCL.to('kg/m**3'):.2f}")

Wing loading: 5.19 kg/m**3


| Type of Aircraft | WCL (kg/m^3) |
| --- | --- |
| Gliders | under 4 |
| Trainers | 5-7 |
| Sport Aerobatic | 8-10 |
| Racers | 11-13 |
| Scale | over 15 |

This could probably be a bit lower. Although my eventual vision is more like a glider-with-enough-power-for-autonomous-takeoffs type design, so I guess it's not surprising I won't have the most incredible glide slope.

In [17]:
TtoW = 1.1
T = TtoW*W
print(f"Required thrust: {T.to('kg'):.2f}")

Required thrust: 3.35 kg


## Propulsion system

Refer back to iteration 3 for kv and prop pitch Y estimates as we did not change the airspeed, so the required kv ratings and pitch Y estimates stay the same. 

Increasing the weight did increase the required thrust from 2.6 to 3.35kg.

### Propulsion system component selection

Looking back at the options I outlined in iteration 3, I couldn't find anything which can sustain the 3.35kg required thrust. I decided to forgo that requirement. A 16x10 inch prop at full throttle may not be sustainable continuously for the [`PROPDRIVE v2 4258 500KV`](https://hobbyking.com/en_us/propdrive-v2-4258-500kv-brushless-outrunner-motor.html), but I think it'll be fine because:

- This would only occur at takeoff
- The manufacturer published the 16x10 test results so at least it's not *completely* fatal to run it like that
- rcplanes.online's excellent [Electric Motor & Prop Combo Estimator](https://rcplanes.online/calc_motor.htm) indicates the 16x10/500Kv/6S combination should be able to generate 3.5kg of thrust @ 46.7A (below the Propdrive's 60A max). I probably chose a bunch of paramters wrong, but still, it indicates the current draw will not be completely ludicrous.

In conclusion, I will be keeping the selction I made in iteration 3:  
Motor: [`PROPDRIVE v2 4258 500KV Brushless Outrunner Motor`](https://hobbyking.com/en_us/propdrive-v2-4258-500kv-brushless-outrunner-motor.html) - €46.10  
Prop: [`TGS Precision Sport Propeller 17x10`](https://hobbyking.com/en_us/tgs-precision-sport-propeller-17x10-1pc.html) - €6.14 * 3 = €18.42  
ESC: [`YEP 80A (2~6S) SBEC`](https://hobbyking.com/en_us/yep-80a-2-6s-sbec-brushless-speed-controller.html) - €43.49  
Battery: [`Turnigy nano-tech 4000mAh 6S 35~70C`](https://hobbyking.com/en_us/turnigy-nano-tech-4000mah-6s-35-70c-lipo-pack-w-xt-90.html) - €63.39  
Total power system: €171.40